In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import kurtosis, skew
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scikit_posthocs as sp
from scipy.stats import wilcoxon

In [3]:
tempTopDir = '/Volumes/Britt_SSD/ReDoEvals3/UserStudy/'
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')

filePath = os.path.join(tempTopDir, 'likertSlim.parquet')
likertSlim = pd.read_parquet(filePath)


likertSlim['visuals'] = np.where(likertSlim['visuals']=="Visuals","Charts",
                        np.where(likertSlim['visuals']=="Blur_Plus_Vis","Blur+Charts",likertSlim['visuals']))

# likertSlim['ag_correctedPerc'] = likertSlim['1_2_correctedPerc']
# likertSlim['Q17_Z'] = likertSlim['17.0_Z']
# likertSlim['Q18_Z'] = likertSlim['18.0_Z']
# likertSlim['Q19_Z'] = likertSlim['19.0_Z']
# likertSlim['Q20_Z'] = likertSlim['20.0_Z']

In [4]:
# drop particpants who have not been accepted (rejected or timed out)
pID_removed1 = likertSlim[(likertSlim['status']!='APPROVED')].ParticipantPublicID.unique()
print(len(likertSlim))
# drop these given row
# indexes from dataFrame
likertSlim = likertSlim[~likertSlim['ParticipantPublicID'].isin(pID_removed1)].copy()
print(len(likertSlim))
# tmp_slimDF = slimDF[~slimDF['ParticipantPublicID'].isin(pID_removed1)].copy()
# print(tmp_slimDF.status.unique())
# print(pID_removed1)
    
# tmp_demoDF_L = demoDF_L[~demoDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_demoDF_S = demoDF_S[~demoDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_consentDF_L = consentDF_L[~consentDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_consentDF_S = consentDF_S[~consentDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_aiDF_L = aiDF_L[~aiDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_aiDF_S = aiDF_S[~aiDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_taskDF = taskDF[~taskDF['ParticipantPublicID'].isin(pID_removed1)].copy()

# print(tmp_taskDF[tmp_taskDF['ParticipantPublicID']==pID_removed1[-3]])

233
219


In [5]:
df = likertSlim.copy()

for vis in df.visuals.unique():
    print(str(vis) + " " + str(len(df[df['visuals']==vis])))

Blur+Charts 59
Charts 51
Blurred 53
Video 56


In [6]:
vis = sorted(df['visuals'].unique())
print(vis)

['Blur+Charts', 'Blurred', 'Charts', 'Video']


In [7]:
# function to make statsmodel into dataframe
def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    print(dir(results))
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })

    #Reordering...
    results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
    return results_df

In [8]:
print(likertSlim.columns.to_list())

['ParticipantPrivateID', 'difficulty', 'visuals', 'agents', 'QsTotal', 'QsCorrect', 'Agent_QsTotal', 'Agent_QsCorrect', 'Agent_QsWO_GhostsTotal', 'Agent_QsWO_GhostsCorrect', 'Qs1_Correct', 'Qs2_Correct', 'QsPart_Total', 'predictionsCorrect', 'predictionsTotal', 'predictions1Correct', 'predictions2Correct', 'predictionsHalfTotal', 'regionsCorrect', 'regionsTotal', 'regions1Correct', 'regions2Correct', 'regionsHalfTotal', 'levelCorrect', 'levelTotal', 'level1Correct', 'level2Correct', 'levelHalfTotal', 'ghostsCorrect', 'ghostsTotal', 'ghosts1Correct', 'ghosts2Correct', 'ghostsHalfTotal', 'comparisonsCorrect', 'comparisonsTotal', 'timeTakenMain_All', 'timeTakenMain_AgentsOnly', 'timeTaken_Consent', 'timeTaken_Demographics', 'timeTaken_AI', 'timeTaken_Agent1', 'timeTaken_Agent2', 'timeTaken_Agent3', 'time_TOTAL', 'vidPlayed', 'avgVidPlays', 'ratioVidPlaysToAvg', 'ExperimentVersion_x', 'totPerc', 'agTotPerc', 'tot1Perc', 'tot2Perc', 'Agent_QsWO_GhostsPerc', 'predPerc', 'pred1Perc', 'pred2Pe

In [9]:
def negative_bold(val):
    bold = 'bold' if val < 0 else ''
    return 'font-weight: %s' % bold

In [10]:
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
# Part Three
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################
######################################################################################



In [11]:
# # Note that the three terms returned are X² statistic, p-value, and degree of freedom
def testChiCorr(a,b,c):
    ct = pd.crosstab(c[a], c[b], margins=True)
    ar = []
    for i in range(0, len(ct)-1):
        ar.append(list(ct.iloc[i][0:-1].values))
    obs = np.array(ar)
    X, p, degF = stats.chi2_contingency(obs)[0:3]
    if (p<0.05):
        print("\n")
        print(str(a) + " " + str(stats.tmean(c[a])) + " " + str(stats.tstd(c[a])))
        print(str(b) + " " + str(stats.tmean(c[b])) + " " + str(stats.tstd(c[b])))
#     else:
#         print("\n"+ str(a) + " " + str(b))
    return(X, p, degF)

In [12]:
# Part Three:

# ORDINAL
df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Original_Rating'
b = 'Car_Explainable_Rating'
print("More explainable cars")
print(testChiCorr(a,b,df))

# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Original_Rating'
b = 'ai_opinion_quantised'
print("\nAI and liking sd Cars")
print(testChiCorr(a,b,df))

# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Explainable_Rating'
b = 'ai_opinion_quantised'
print("\nAI and preferring explainable Cars")
print(testChiCorr(a,b,df))

# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Original_Rating'
b = 'age_info_quantised'
print("\nAge and sd Cars")
print(testChiCorr(a,b,df))


# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Explainable_Rating'
b = 'age_info_quantised'
print("\nAge and preferring explainable Cars")
print(testChiCorr(a,b,df))

# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Explainable_Rating'
b = 'age_info_quantised'
print("\nAge and preferring explainable Cars")
print(testChiCorr(a,b,df))

#gender_id_quantised
print(df['gender_id_quantised'].unique())
df['gender_id_quantised'] = df['gender_id_quantised'].fillna(value=0)
print(df['gender_id_quantised'].unique())
# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Original_Rating'
b = 'gender_id_quantised'
print("\nGender and sd Cars")
print(testChiCorr(a,b,df))
# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Explainable_Rating'
b = 'gender_id_quantised'
print("\nGender and preferring explainable Cars")
print(testChiCorr(a,b,df))

More explainable cars


Car_Original_Rating 3.34703196347032 1.2146384873438487
Car_Explainable_Rating 4.054794520547945 0.7937325184188475
(70.27558302944387, 8.928919633159756e-09, 16)

AI and liking sd Cars
(20.39168274299768, 0.20309511648548678, 16)

AI and preferring explainable Cars


Car_Explainable_Rating 4.054794520547945 0.7937325184188475
ai_opinion_quantised 3.6027397260273974 0.8790068315595534
(42.635966819656645, 0.00031697401440959226, 16)

Age and sd Cars
(25.597282296377195, 0.3739099528959618, 24)

Age and preferring explainable Cars
(21.14402161181426, 0.6302150264186277, 24)

Age and preferring explainable Cars
(21.14402161181426, 0.6302150264186277, 24)
[ 2.  1. nan  3.]
[2. 1. 0. 3.]

Gender and sd Cars


Car_Original_Rating 3.34703196347032 1.2146384873438487
gender_id_quantised 1.5068493150684932 0.5278471336058865
(23.112632639424742, 0.026784803952831653, 12)

Gender and preferring explainable Cars
(5.51855597910644, 0.9383843696917461, 12)


In [15]:
# Part Three:

# ORDINAL
df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()

# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Original_Rating'
b = 'totPerc'
print("More explainable cars")
print(testChiCorr(a,b,df))

# Chi-Square on likert1Mode v tot1Perc
a = 'totPerc'
b = 'ai_opinion_quantised'
print("\nAI and liking sd Cars")
print(testChiCorr(a,b,df))

# Chi-Square on likert1Mode v tot1Perc
a = 'Car_Explainable_Rating'
b = 'totPerc'
print("\nAI and preferring explainable Cars")
print(testChiCorr(a,b,df))

# Chi-Square on likert1Mode v tot1Perc
a = 'totPerc'
b = 'age_info_quantised'
print("\nAge and sd Cars")
print(testChiCorr(a,b,df))



#gender_id_quantised
print(df['gender_id_quantised'].unique())
df['gender_id_quantised'] = df['gender_id_quantised'].fillna(value=0)
print(df['gender_id_quantised'].unique())
# Chi-Square on likert1Mode v tot1Perc
a = 'totPerc'
b = 'gender_id_quantised'
print("\nGender and sd Cars")
print(testChiCorr(a,b,df))

More explainable cars
(47.50039127901292, 0.7834555211407273, 56)

AI and liking sd Cars
(50.554557329341414, 0.680346348134365, 56)

AI and preferring explainable Cars
(71.40785913407781, 0.08043794856114488, 56)

Age and sd Cars
(63.67395193619157, 0.9519074171827786, 84)
[ 2.  1. nan  3.]
[2. 1. 0. 3.]

Gender and sd Cars
(38.45144545245882, 0.6275269171629065, 42)


In [32]:
# Part Three:

#'Car_Original_Response', 'Car_Explainable_Response'

# Logistic
pVal = 0.01
rAdj = 0.05

df = likertSlim.copy()
# df = likertSlim[likertSlim['difficulty']=="Easy"].copy()
# df = likertSlim[likertSlim['difficulty']=="Difficult"].copy()
df['Car_Original_Response']=pd.Categorical(df['Car_Original_Response'])

# Logistic Regression on likert1Mode v tot1Perc
f_rev1 = 'tot1Perc ~ C(Car_Original_Response)'
est = smf.ols(formula=f_rev1, data=df).fit()
if est.f_pvalue < 0.05 and est.rsquared_adj > rAdj:
    print('Summary: \n', est.summary())

# Logistic Regression on likert1Mode v tot2Perc
f_rev1 = 'tot1Perc ~ C(Car_Explainable_Response)'
est = smf.ols(formula=f_rev1, data=df).fit()
if est.f_pvalue < 0.05 and est.rsquared_adj > rAdj:
    print('Summary: \n', est.summary())

# Logistic Regression on likert1Mode v tot2Perc
f_rev1 = 'Car_Explainable_Rating ~ Car_Original_Rating'
est = smf.ols(formula=f_rev1, data=df).fit()
if est.f_pvalue < 0.05 and est.rsquared_adj > rAdj:
    print('Summary: \n', est.summary())


Summary: 
                               OLS Regression Results                              
Dep. Variable:     Car_Explainable_Rating   R-squared:                       0.268
Model:                                OLS   Adj. R-squared:                  0.265
Method:                     Least Squares   F-statistic:                     79.51
Date:                    Mon, 05 Jul 2021   Prob (F-statistic):           2.01e-16
Time:                            12:59:29   Log-Likelihood:                -225.47
No. Observations:                     219   AIC:                             454.9
Df Residuals:                         217   BIC:                             461.7
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------